<a href="https://colab.research.google.com/github/tcheskko/TimeSeries_OceanAtmo/blob/main/DataPrepar_INMET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from google.colab import files
uploaded = files.upload()

Saving INMET_A806_FLORIANOPOLIS_2018.CSV to INMET_A806_FLORIANOPOLIS_2018.CSV
Saving INMET_A806_FLORIANOPOLIS_2019.CSV to INMET_A806_FLORIANOPOLIS_2019.CSV
Saving INMET_A806_FLORIANOPOLIS_2020.CSV to INMET_A806_FLORIANOPOLIS_2020.CSV
Saving INMET_A806_FLORIANOPOLIS_2021.CSV to INMET_A806_FLORIANOPOLIS_2021.CSV
Saving INMET_A806_FLORIANOPOLIS_2022.CSV to INMET_A806_FLORIANOPOLIS_2022.CSV


In [3]:
met2018 = pd.read_csv("INMET_A806_FLORIANOPOLIS_2018.CSV", encoding='ISO-8859-1', sep=';')
met2019 = pd.read_csv("INMET_A806_FLORIANOPOLIS_2019.CSV", encoding='ISO-8859-1', sep=';')
met2020 = pd.read_csv("INMET_A806_FLORIANOPOLIS_2020.CSV", encoding='ISO-8859-1', sep=';')
met2021 = pd.read_csv("INMET_A806_FLORIANOPOLIS_2021.CSV", encoding='ISO-8859-1', sep=';')
met2022 = pd.read_csv("INMET_A806_FLORIANOPOLIS_2022.CSV", encoding='ISO-8859-1', sep=';')

In [4]:
framesINM = [met2018,met2019,met2020,met2021,met2022]
metFULL = pd.concat(framesINM)

In [5]:
print(metFULL.shape)
print( metFULL.columns)

(43824, 19)
Index(['DATA (YYYY-MM-DD)', 'HORA', 'PREC_TOTAL', 'PATM_EST', 'PATM_MAX',
       'PATM_MIN', 'RAD', 'TEMP_BSE', 'TEMP_ORV', 'TEMP_MAX', 'TEMP_MIN',
       'TEMP_ORV_MAX', 'TEMP_ORV_MIN', 'UMID_MAX', 'UMID_MIN', 'UMID_AR',
       'VENT_DIR', 'VENT_RAJ', 'VENT_VEL'],
      dtype='object')


## Remoção de colunas que não serão úteis

In [6]:
#Removendo colunas que nao serão usadas

del metFULL["PATM_MAX"]
del metFULL["PATM_MIN"]
del metFULL["RAD"]
del metFULL["TEMP_MAX"]
del metFULL["TEMP_MIN"]
del metFULL["TEMP_ORV_MAX"]
del metFULL["TEMP_ORV_MIN"]
del metFULL["UMID_MAX"]
del metFULL["UMID_MIN"]
del metFULL["PATM_EST"]
del metFULL["TEMP_ORV"]
del metFULL["VENT_RAJ"]


In [7]:
print(metFULL.shape)
print( metFULL.columns)

(43824, 7)
Index(['DATA (YYYY-MM-DD)', 'HORA', 'PREC_TOTAL', 'TEMP_BSE', 'UMID_AR',
       'VENT_DIR', 'VENT_VEL'],
      dtype='object')


In [8]:
metFULL.dtypes

DATA (YYYY-MM-DD)     object
HORA                  object
PREC_TOTAL            object
TEMP_BSE              object
UMID_AR              float64
VENT_DIR             float64
VENT_VEL              object
dtype: object

## Ajuste da coluna 'DATA'

In [9]:
#Renomeando a coluna 'data'

metFULL.rename(columns={'DATA (YYYY-MM-DD)': 'DATE'}, inplace=True)

#Modificando para DateTime a coluna 'Data'

metFULL['DATE'] = pd.to_datetime(metFULL['DATE'])

<ipython-input-9-28fcc9c131ef>:7: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  metFULL['DATE'] = pd.to_datetime(metFULL['DATE'])


In [10]:
metFULL.dtypes

DATE          datetime64[ns]
HORA                  object
PREC_TOTAL            object
TEMP_BSE              object
UMID_AR              float64
VENT_DIR             float64
VENT_VEL              object
dtype: object

## Tratamento de valores inconsistentes
- Ajuste do separador de milhar para float
- Busca por dados nulos
- Calculo de média de dados de dias anteriores para gerar novo valor

In [11]:
#Buscando valores nulos

print(metFULL.isnull().sum())

DATE            0
HORA            0
PREC_TOTAL    750
TEMP_BSE       29
UMID_AR        29
VENT_DIR       29
VENT_VEL       29
dtype: int64


In [12]:
#Transformando dados para float (somente objects) e trocando a virgula por ponto

metFULL['PREC_TOTAL'] = pd.to_numeric(metFULL['PREC_TOTAL'].str.replace(',','.'), errors='ignore').astype(pd.Float64Dtype())
metFULL['TEMP_BSE'] = pd.to_numeric(metFULL['TEMP_BSE'].str.replace(',','.'), errors='ignore').astype(pd.Float64Dtype())
metFULL['VENT_VEL'] = pd.to_numeric(metFULL['VENT_VEL'].str.replace(',','.'), errors='ignore').astype(pd.Float64Dtype())

In [13]:
metFULL.dtypes

DATE          datetime64[ns]
HORA                  object
PREC_TOTAL           Float64
TEMP_BSE             Float64
UMID_AR              float64
VENT_DIR             float64
VENT_VEL             Float64
dtype: object

In [14]:
#Encontrando valores nulos

print(metFULL.isnull().sum())

DATE            0
HORA            0
PREC_TOTAL    750
TEMP_BSE       29
UMID_AR        29
VENT_DIR       29
VENT_VEL       29
dtype: int64


In [15]:
metFULL["VENT_VEL"]

0        1.2
1        0.9
2        0.5
3        0.2
4        0.7
        ... 
8755    <NA>
8756    <NA>
8757    <NA>
8758    <NA>
8759    <NA>
Name: VENT_VEL, Length: 43824, dtype: Float64

In [16]:
#Em cada linha onde a coluna 'N' é nula, calculo a média dos valores das 10 linhas anteriores a essa linha nula

for idx, row in metFULL.iterrows():
  if pd.isnull(row["VENT_VEL"]):
    valuesAVG = metFULL.loc[idx, "VENT_VEL"].head(10)
    valuesNOT_NULL = valuesAVG.dropna()
    avr_VENT_VEL = np.median(valuesNOT_NULL)
    metFULL.at[idx, "VENT_VEL"] = avr_VENT_VEL

In [17]:
print(metFULL.isnull().sum())

DATE            0
HORA            0
PREC_TOTAL    750
TEMP_BSE       29
UMID_AR        29
VENT_DIR       29
VENT_VEL        0
dtype: int64


In [18]:
for idx, row in metFULL.iterrows():
  if pd.isnull(row["VENT_DIR"]):
    valuesToAVG = metFULL.loc[idx, "VENT_DIR"].head(10).dropna()
    newAVG = np.median(valuesToAVG)
    metFULL.at[idx, "VENT_DIR"] = newAVG

In [19]:
for idx, row in metFULL.iterrows():
  if pd.isnull(row["UMID_AR"]):
    valuesToAVG = metFULL.loc[idx, "UMID_AR"].head(10).dropna()
    newAVG = np.median(valuesToAVG)
    metFULL.at[idx, "UMID_AR"] = newAVG

In [20]:
for idx, row in metFULL.iterrows():
  if pd.isnull(row["TEMP_BSE"]):
    valuesToAVG = metFULL.loc[idx, "TEMP_BSE"].head(10).dropna()
    newAVG = np.median(valuesToAVG)
    metFULL.at[idx, "TEMP_BSE"] = newAVG

In [21]:
for idx, row in metFULL.iterrows():
  if pd.isnull(row["PREC_TOTAL"]):
    valuesToAVG = metFULL.loc[idx, "PREC_TOTAL"].head(10).dropna()
    newAVG = np.median(valuesToAVG)
    metFULL.at[idx, "PREC_TOTAL"] = newAVG

In [22]:
print(metFULL.isnull().sum())

DATE          0
HORA          0
PREC_TOTAL    0
TEMP_BSE      0
UMID_AR       0
VENT_DIR      0
VENT_VEL      0
dtype: int64


In [23]:
metFULL.shape

(43824, 7)

In [24]:
metFULL.dtypes

DATE          datetime64[ns]
HORA                  object
PREC_TOTAL           Float64
TEMP_BSE             Float64
UMID_AR              float64
VENT_DIR             float64
VENT_VEL             Float64
dtype: object

## Ajuste das colunas 'DATA' e 'HORA'
- Formatação da data;
- Ajuste do tipo de dado para 'HORA'
- Remoção de linhas não úteis a partir do dado 'HORA'
- Remoção da coluna 'HORA'

In [25]:
from datetime import datetime, timezone

In [26]:
#Remove todas as letras da coluna
metFULL['HORA'] = metFULL['HORA'].str.replace('\D', '', regex=True)

In [27]:
print(metFULL.isnull().sum())

DATE          0
HORA          0
PREC_TOTAL    0
TEMP_BSE      0
UMID_AR       0
VENT_DIR      0
VENT_VEL      0
dtype: int64


In [28]:
metFULL['HORA'] = pd.to_numeric(metFULL['HORA'])

In [29]:
metFULL["HORA"]

0          0
1        100
2        200
3        300
4        400
        ... 
8755    1900
8756    2000
8757    2100
8758    2200
8759    2300
Name: HORA, Length: 43824, dtype: int64

In [30]:
metFULL.shape

(43824, 7)

In [31]:
print(metFULL.isnull().sum())

DATE          0
HORA          0
PREC_TOTAL    0
TEMP_BSE      0
UMID_AR       0
VENT_DIR      0
VENT_VEL      0
dtype: int64


In [32]:
metFULL.drop(metFULL[(metFULL['HORA'] >1200)].index, inplace=True)

In [33]:
metFULL.shape

(23738, 7)

In [34]:
metFULL.drop(metFULL[(metFULL['HORA'] <1200)].index, inplace=True)

In [35]:
metFULL.shape

(1826, 7)

In [36]:
metFULL["HORA"]

12      1200
36      1200
60      1200
84      1200
108     1200
        ... 
8652    1200
8676    1200
8700    1200
8724    1200
8748    1200
Name: HORA, Length: 1826, dtype: int64

In [37]:
metFULL['DATE'] = pd.to_datetime(metFULL["DATE"].dt.strftime('%d-%m-%y'))

In [38]:
del metFULL["HORA"]

## Nova coluna 'VENT_DROS'
Direção do vento convertida de angulos (coluna 'VENT_DIR') para texto (Rosa dos ventos)

In [39]:
for idx, row in metFULL.iterrows():
  if pd.isnull(row["VENT_DIR"]) == False:
    ang_VDIR = row["VENT_DIR"]
    new_VDIR_CR = int((ang_VDIR/22.5)+.5)
    arr_CR = ["N","NNE","NE","ENE","E","ESE", "SE", "SSE","S","SSW","SW","WSW","W","WNW","NW","NNW"]
    metFULL.loc[idx,"VENT_DROS"] = arr_CR[(new_VDIR_CR % 16)]

In [40]:
print(metFULL.isnull().sum())

DATE          0
PREC_TOTAL    0
TEMP_BSE      0
UMID_AR       0
VENT_DIR      0
VENT_VEL      0
VENT_DROS     0
dtype: int64


## Alteração do nome das colunas para inserir prefixo 'INM_'

In [41]:
metFULL.rename(columns={'PREC_TOTAL': 'INM_PREC_TOTAL'}, inplace=True)
metFULL.rename(columns={'TEMP_BSE': 'INM_TEMP_BSE'}, inplace=True)
metFULL.rename(columns={'UMID_AR': 'INM_UMID_AR'}, inplace=True)
metFULL.rename(columns={'VENT_DIR': 'INM_VENT_DIR'}, inplace=True)
metFULL.rename(columns={'VENT_VEL': 'INM_VENT_VEL'}, inplace=True)
metFULL.rename(columns={'VENT_DROS': 'INM_VENT_DROS'}, inplace=True)

In [42]:
metFULL.columns

Index(['DATE', 'INM_PREC_TOTAL', 'INM_TEMP_BSE', 'INM_UMID_AR', 'INM_VENT_DIR',
       'INM_VENT_VEL', 'INM_VENT_DROS'],
      dtype='object')